In [2]:
!gdown --id 1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu

/media/tan/F/AIO2024_hw/env/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu
To: /media/tan/F/AIO2024_hw/AIO2024_hw/Module5/week3/Auto_MPG_data.csv
100%|███████████████████████████████████████| 15.4k/15.4k [00:00<00:00, 151kB/s]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_state)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
dataset_path = 'Auto_MPG_data.csv'
dataset = pd.read_csv(dataset_path)
dataset

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504.0,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,0,0,1
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790.0,15.6,82,0,0,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,1,0,0
389,32.0,4,135.0,84.0,2295.0,11.6,82,0,0,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,0,0,1


In [6]:
# Preprocessing
X = dataset.drop(columns=['MPG']).values
y = dataset['MPG'].values

In [7]:
val_size = 0.2
test_size = 0.15
is_shuffle = True
random_state = 50

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=random_state, shuffle=is_shuffle)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=test_size, random_state=random_state, shuffle=is_shuffle)

In [8]:
# Normalization input features
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

# Convert to tensor
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [9]:
# DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# Build MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = F.sigmoid(x)
        out = self.output(x)
        x = F.sigmoid(x)
        return out.squeeze(1)

In [16]:
input_dim = X_train.shape[1]
output_dim = 1
hidden_dim = 64

model = MLP(input_dim, hidden_dim, output_dim).to(device)

In [17]:
# loss and optimizer
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [18]:
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [19]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

for epoch in range(epochs):
    train_loss = 0.0
    train_target, train_predict = [], []
    val_target, val_predict = [], []

    # Training phase
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        
        train_predict += outputs.tolist()
        train_target += y_batch.tolist()

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            val_predict += outputs.tolist()
            val_target += y_batch.tolist()
            
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))

    print(f"\nEPOCH {epoch + 1}:\tTraining loss: {train_loss:.3f}\tValidation loss: {val_loss:.3f}")



EPOCH 1:	Training loss: 146.608	Validation loss: 6.883

EPOCH 2:	Training loss: 18.411	Validation loss: 5.037

EPOCH 3:	Training loss: 15.566	Validation loss: 4.695

EPOCH 4:	Training loss: 14.772	Validation loss: 5.340

EPOCH 5:	Training loss: 14.446	Validation loss: 4.020

EPOCH 6:	Training loss: 13.624	Validation loss: 4.254

EPOCH 7:	Training loss: 13.207	Validation loss: 4.103

EPOCH 8:	Training loss: 13.278	Validation loss: 3.844

EPOCH 9:	Training loss: 13.178	Validation loss: 4.010

EPOCH 10:	Training loss: 12.724	Validation loss: 3.925

EPOCH 11:	Training loss: 12.518	Validation loss: 3.832

EPOCH 12:	Training loss: 12.297	Validation loss: 3.590

EPOCH 13:	Training loss: 12.212	Validation loss: 3.768

EPOCH 14:	Training loss: 12.046	Validation loss: 3.969

EPOCH 15:	Training loss: 12.087	Validation loss: 4.115

EPOCH 16:	Training loss: 11.944	Validation loss: 3.764

EPOCH 17:	Training loss: 11.698	Validation loss: 3.843

EPOCH 18:	Training loss: 11.650	Validation loss: 3.910


In [20]:
model.eval()
with torch.no_grad() :
    y_hat = model(X_test)
    test_set_r2 = r_squared(y_hat,y_test)
    print('Evaluation on test set :')
    print(f'R2: {test_set_r2}')

Evaluation on test set :
R2: 0.8859344124794006
